In [7]:
import numpy
import rioxarray
import xarray
import geopandas
import pathlib
import netCDF4

# Small examples of creating an xarray from numpy arrays

In [2]:
base_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\DEMs\example_nc_file")

In [ ]:
raster_origin = [10, 35]
raster_size = [200, 100]
resolution = 10
crs = 2193

x_coords = numpy.arange(raster_origin[0], raster_origin[0] + resolution * raster_size[0], resolution)
y_coords = numpy.arange(raster_origin[1] + resolution * raster_size[1],
                        raster_origin[1], -resolution) # following convention of GDAL where y starts at the largest
z_array = numpy.random.rand(1, len(y_coords), len(x_coords)) # 1 is the band dimension

example_xarray = xarray.DataArray(z_array,
                                  coords = {'band': [1], 'x': x_coords, 'y': y_coords},
                                  dims = ['band', 'y','x'], 
                                  attrs= {'scale_factor': 1.0,
                                          'add_offset': 0.0,
                                          'long_name': 'idw',
                                          '_FillValue': numpy.nan})
example_xarray.rio.write_crs(crs, inplace=True)
example_xarray.rio.write_transform(inplace=True)
example_xarray.to_netcdf(base_path / "data_array_ex.nc")

In [ ]:
print(example_xarray.rio.transform(), '\n', example_xarray.rio.crs)

In [ ]:
with xarray.open_dataset(base_path / "data_array_ex.nc") as ex_data:
    ex_data.load()
    print(ex_data.rio.transform(), '\n', ex_data.rio.crs)
#ex_data = ex_data.copy(deep=True)

In [ ]:
with rioxarray.rioxarray.open_rasterio(base_path / "data_array_ex.nc", masked=True) as df:
    df.load()
    print(df.rio.transform(), '\n', df.rio.crs)
#df = df.copy(deep=True)

### How to load in crs using gdal and pyproj

In [ ]:
import osgeo.gdal
import pyproj
ds_gdal = osgeo.gdal.Open(str(base_path / "data_array_ex.nc"))
pyproj.CRS.from_string(ds_gdal.GetProjection())

# Example from Cyprien

In [ ]:
ex_path = pathlib.Path(r"C:\Users\pearsonra\Documents\data\DEMs\example_nc_file") \
    / 'Adaptdemo_coarse.nc'

In [ ]:
with rioxarray.rioxarray.open_rasterio(ex_path, masked=True) as ex_data:
    ex_data.load()
ex_data = ex_data.copy(deep=True)

In [ ]:
print(ex_data.rio.transform(), '\n', ex_data.rio.crs)

In [ ]:
ex_data

# Creating a DataSet from numpy arrays

In [8]:
origin = [10, 35]
size = [200, 100]
resolutions = [5, 10]
crs = 2193

for resolution in resolutions:
    x = numpy.arange(origin[0],
                     origin[0] + resolution * size[0],
                        resolution)
    y = numpy.arange(origin[1] + resolution * size[1],
                     origin[1],
                     -resolution) # following convention of GDAL where y starts at the largest
    elevation = numpy.random.rand(len(x), len(y))
    roughness = numpy.random.rand(len(x), len(y))
    category = numpy.random.randint(0, 4, (len(x), len(y)))

    example_xarray = xarray.Dataset(
        data_vars=dict(z=(["x", "y"], elevation, {"units": "m", "vertical_datum": "EPSG:7839", "long_name": "ground elevation"}),
                       zo=(["x", "y"], roughness, {"units": "", "long_name": "roughness length"}),
                       source=(["x", "y"], category, {"units": "int", "long_name": "Data source. 0=LiDAR, 1=Ocean bathymetry, 2=River bathymetry, 3=interpolated"})),
        coords=dict(
            x=(["x"], x, {"long_name": "Easting", "units": "m"}),
            y=(["y"], y)),
    attrs= {"title": "GeoFabrics representing elevation and roughness",
            "source": "GeoFabrics 0.8.1",
            "history": "append; date create GeoFabrics:processor_class",
            "description": f"GeoFabrics:processor_class resolution xx",
            "geofabrics_instructions": "dumped instruction file"})
        
    # Ensure the CRS and transform are respectively written in CF 
    # complient manner, and 'where GDAL can read it'
    example_xarray.rio.write_crs(crs, inplace=True)
    example_xarray.z.rio.write_crs(crs, inplace=True)
    example_xarray.zo.rio.write_crs(crs, inplace=True)
    example_xarray.source.rio.write_crs(crs, inplace=True)
    example_xarray.rio.write_transform(inplace=True)
    # Write to a grouped netCDF file and a stand alone file for just this grouping
    mode = "a" if (base_path / "grouped_data.nc").exists() else "w"
    example_xarray.to_netcdf(base_path / "grouped_data.nc",
                             group=f"/resolution_{resolution}",
                             mode=mode,
                             engine="netcdf4")
    example_xarray.to_netcdf(base_path / f"data_resolution_{resolution}.nc")

In [13]:
example_xarray.z.data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
example_xarray

In [ ]:
example_xarray.coords['spatial_ref']

In [ ]:
example_xarray.rio.clip()

In [ ]:
example_xarray.rio.clip()

In [ ]:
print(example_xarray.rio.transform(), '\n', example_xarray.rio.crs)

In [ ]:
with rioxarray.rioxarray.open_rasterio(base_path / "data_resolution_5.nc", masked=True) as ex_data:
    ex_data.load()
    print(ex_data.rio.transform(), '\n', ex_data.rio.crs)

In [ ]:
with rioxarray.rioxarray.open_rasterio(base_path / "grouped_data.nc", group="/resolution_10", masked=True) as ex_data:
    ex_data.load()
    print(ex_data.rio.transform(), '\n', ex_data.rio.crs)

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_5") as ds_5:
    #print(ds_5.rio.transform(), '\n', ds_5.rio.crs)
    ds_5.rio.set_crs(int((ds_5.attrs['crs'])), inplace=True);
    ds_5.roughness.plot()
    print(ds_5.rio.transform(), '\n', ds_5.rio.crs)

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_5") as ds_5:
    print(ds_5.rio.transform(), '\n', ds_5.rio.crs)
    ds_5.rio.set_crs(int((ds_5.attrs['crs'])), inplace=True);
    ds_5.roughness.plot()
    print(ds_5.rio.transform(), '\n', ds_5.rio.crs)

In [ ]:
with xarray.open_dataset(base_path / "grouped_data.nc", group="/resolution_10") as ds_10:
    ds_10.rio.set_crs(int((ds_5.attrs['crs'])), inplace=True);
    ds_10.roughness.plot()
    print(ds_10.rio.transform(), '\n', ds_10.rio.crs)

In [ ]:
import osgeo.gdal
import pyproj
ds_gdal = osgeo.gdal.Open(str(base_path / "data_resolution_5.nc"))
crs=pyproj.CRS.from_string(ds_gdal.GetProjection())

with xarray.open_dataset(base_path / "data_resolution_5.nc") as ds_5:
    
    ds_5.rio.set_crs(crs, inplace=True);
    ds_5.roughness.plot()
    print(ds_5.rio.transform(), '\n', ds_5.rio.crs)